# Imports

In [1]:
import pandas as pd
import numpy as np

# Load data

## 2001

In [2]:
religions = ['Christian', 'Buddhist', 'Hindu', 'Jewish', 'Muslim', 'Sikh', 'Other religion', 'No religion', 'Religion not stated']

sex = ['male', 'female']

In [3]:
dfs = []
skip = 9
counter = 0
for s in sex:
    for r in religions:
        skiprows = skip + (counter * 371)
        df = pd.read_csv('../raw_data/ew/census_2001_ST149_religion_sex_5yrband.csv', skiprows=skiprows).iloc[:348]
        df['sex'] = s
        df['religion'] = r
        dfs.append(df)
        counter += 1

df_result = pd.concat(dfs).reset_index(drop=True)

In [4]:
cols = list(df_result)
cols[0:2] = ['geography', 'geo_code']
df_result.columns = cols

In [5]:
df_result.iloc[:, 2:-2] = df_result.iloc[:, 2:-2].astype(int)

In [6]:
df_2001 = df_result.melt(
    id_vars=['geography', 'geo_code', 'sex', 'religion'],
    var_name='age',
    value_name='population'
)

In [7]:
df_2001['census_year'] = 2001

In [8]:
lower = (df_2001['age'].str.split().str[1].astype(int) // 5 * 5)
upper = lower + 4
df_2001['age_band'] = lower.astype(str).str.cat(upper.astype(str), '-').replace({'90-94': '90+'})
df_2001['yob'] = (2001 - upper).astype(str).str.cat((2001 - lower).astype(str), '-')

In [9]:
df_2001['yob'] = df_2001['yob'].replace({
    '1922-1926': 'pre-1927',
    '1917-1921': 'pre-1927',
    '1912-1916': 'pre-1927',
    '1907-1911': 'pre-1927',
})

In [10]:
# df_2001[['yob', 'age_band']].value_counts()

## 2011

In [11]:
lda_2011 = pd.read_csv('../raw_data/ew/census_2011_localdistrictauthorities2015.csv')

In [12]:
lda_2011 = lda_2011.melt(id_vars=lda_2011.columns[:3])

lda_2011['sex'] = lda_2011['variable'].str.split('[:;]', regex=True).str[1].str.strip()

lda_2011 = lda_2011.loc[lda_2011['sex'].isin(['Males', 'Females'])]

lda_2011['religion'] = lda_2011['variable'].str.split('[:;]', regex=True).str[5].str.strip()

In [13]:
religions = ['Christian', 'Buddhist', 'Hindu', 'Jewish', 'Muslim', 'Sikh', 'Other religion', 'No religion', 'Religion not stated']

In [14]:
lda_2011 = lda_2011.loc[lda_2011['religion'].isin(religions)]

lda_2011['age'] = lda_2011['variable'].str.split('[:;]', regex=True).str[3].str.strip().str[4:].str.replace(' to ', '-').str.replace(' and over', '+')

lda_2011.drop(columns=['variable', 'date'], inplace=True)

## 2021

In [15]:
ew_2021 = pd.read_excel('../raw_data/ew/census_2021_religionagesex.xlsx', sheet_name='Table 3', header=3)

In [16]:
ew_2021 = ew_2021.melt(id_vars=ew_2021.columns[:3])

In [17]:
ew_2021['sex'] = ew_2021['variable'].str.split().str[-1].str.strip()
ew_2021['religion'] = ew_2021['variable'].str.rsplit(n=1).str[0]

In [18]:
ew_2021.drop(columns=['variable'], inplace=True)

## Impute missing values for censored data
Compare local totals to England & Wales total and assign missing population equally by area.

In [19]:
grp_cols = ['Age', 'sex', 'religion']

In [20]:
total_pops = ew_2021[ew_2021['Geography Code'].str[:3] == 'K04'].set_index(grp_cols)['value'].apply(lambda x: 2 if x == 'c' else int(x)).rename('ew_total')

In [21]:
temp = ew_2021[ew_2021['Geography Code'].str[:3].isin(['E06', 'E07', 'E08', 'E09', 'W06'])].copy()
temp['local_total'] = temp['value'].apply(lambda x: 0 if x == 'c' else int(x))
temp['local_n_missing'] = temp['value'].apply(lambda x: True if x == 'c' else False)
local_pops = temp.groupby(grp_cols)[['local_total', 'local_n_missing']].sum()

In [22]:
df_impute = local_pops.join(total_pops)
df_impute['imputed'] = ((df_impute['ew_total'] - df_impute['local_total']) / df_impute['local_n_missing']).replace(np.inf, 0)

In [23]:
ew_2021 = ew_2021.join(df_impute['imputed'], on=grp_cols)
ew_2021.loc[ew_2021['value'] == 'c', 'value'] = ew_2021.loc[ew_2021['value'] == 'c', 'imputed']
ew_2021.drop(columns='imputed', inplace=True)

#### Check population sum

In [24]:
ew_2021[ew_2021['Geography Code'].str[:3].isin(['E06', 'E07', 'E08', 'E09', 'W06'])]['value'].sum()

59598403.99999197

In [25]:
ew_2021[ew_2021['Geography Code'].str[:3] == 'K04']['value'].sum()

59597470.07294832

#### Rename columns

In [26]:
col_map = {
    'Age': 'age',
    'Geography Code': 'geography code',
    'Geography Name': 'geography',
}

ew_2021 = ew_2021.rename(columns=lambda x: col_map.get(x, x))

# Join 2011 and 2021 data

In [27]:
master_df = pd.concat([lda_2011, ew_2021], keys=[2011, 2021]).reset_index(0).rename(columns={'level_0': 'census_year'}).reset_index(drop=True)

### Check for geo codes present in one year but not the other

In [28]:
# gcs = {c for c in set(lda_2011['geography code']) - set(ew_2021['geography code']) if c[:2] == 'E0'}
# master_df.loc[master_df['geography code'].isin(gcs), ['geography code', 'geography']].drop_duplicates()

In [29]:
# gcs = {c for c in set(ew_2021['geography code']) - set(lda_2011['geography code']) if c[:2] == 'E0'}
# master_df.loc[master_df['geography code'].isin(gcs), ['geography code', 'geography']].drop_duplicates()

In [30]:
# master_df[master_df['geography'].str.contains('Westminster')]
# master_df[master_df['geography'].str.contains('Scilly')]

### Map of those codes

In [31]:
gc_map = {
    'E07000150': 'E06000061',
    'E07000151': 'E06000062',
    'E07000152': 'E06000061',
    'E07000153': 'E06000061',
    'E07000154': 'E06000062',
    'E07000155': 'E06000062',
    'E07000156': 'E06000061',
    'E07000201': 'E07000245',
    'E07000204': 'E07000245',
    'E07000205': 'E07000244',
    'E07000206': 'E07000244',
    'E09000033': 'E09000033',
    'E07000004': 'E06000060',
    'E07000005': 'E06000060',
    'E07000006': 'E06000060',
    'E07000007': 'E06000060',
    'E06000028': 'E06000058',
    'E06000053': 'E06000053',
    'E06000029': 'E06000058',
    'E07000048': 'E06000058',
    'E07000049': 'E06000059',
    'E07000050': 'E06000059',
    'E07000051': 'E06000059',
    'E07000052': 'E06000059',
    'E07000053': 'E06000059',
    'E07000190': 'E07000246',
    'E07000191': 'E07000246',
    'E06000053': 'E06000052',
    'E09000033': 'E09000001',
}

In [32]:
master_df['geography code'] = master_df['geography code'].replace(gc_map)

In [33]:
master_df = master_df.groupby(list(master_df.columns.drop('value')))['value'].sum().reset_index()

### Check that all low-level codes are present for both years

In [34]:
not_shared = (
    set(master_df.loc[master_df['census_year'] == 2011, 'geography code'].unique())
    ^ set(master_df.loc[master_df['census_year'] == 2021, 'geography code'].unique())
)

not_shared

{'E12000001',
 'E12000002',
 'E12000003',
 'E12000004',
 'E12000005',
 'E12000006',
 'E12000007',
 'E12000008',
 'E12000009',
 'E92000001',
 'K04000001',
 'W92000004'}

In [35]:
master_df = master_df[~master_df['geography code'].isin(not_shared)].reset_index(drop=True)

In [36]:
master_df['census_year'].value_counts()

census_year
2021    598122
2011    131544
Name: count, dtype: int64

# Map age to 5 year buckets

In [37]:
master_df['age'].unique()

array(['0-4', '10-14', '15', '16-17', '18-19', '20-24', '25-29', '30-34',
       '35-39', '40-44', '45-49', '5-7', '50-54', '55-59', '60-64',
       '65-69', '70-74', '75-79', '8-9', '80-84', '85+', 0, 1, 2, 3, 4, 5,
       6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
       58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74,
       75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91,
       92, 93, 94, 95, 96, 97, 98, 99, '100 and over'], dtype=object)

In [38]:
master_df['age_band'] = master_df['age'].apply(lambda x: x if type(x) == str else f'{(x // 5) * 5}-{(x // 5) * 5 + 4}')

In [39]:
master_df['age_band'].unique()

array(['0-4', '10-14', '15', '16-17', '18-19', '20-24', '25-29', '30-34',
       '35-39', '40-44', '45-49', '5-7', '50-54', '55-59', '60-64',
       '65-69', '70-74', '75-79', '8-9', '80-84', '85+', '5-9', '15-19',
       '85-89', '90-94', '95-99', '100 and over'], dtype=object)

In [40]:
ab_map = {
    '5-7': '5-9',
    '8-9': '5-9',
    '15': '15-19',
    '16-17': '15-19',
    '18-19': '15-19',
    '95-99': '95+',
    '100 and over': '95+',
}

In [41]:
master_df['age_band'] = master_df['age_band'].replace(ab_map)

In [42]:
master_df['age_band'].unique()

array(['0-4', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
       '40-44', '45-49', '5-9', '50-54', '55-59', '60-64', '65-69',
       '70-74', '75-79', '80-84', '85+', '85-89', '90-94', '95+'],
      dtype=object)

# Calculate year of birth column

In [43]:
to_age = master_df['age_band'].str.split('[-+]', regex=True).str[1]
to_age.loc[to_age == ''] = 120

from_yob = master_df['census_year'] - to_age.astype(int)

to_yob = master_df['census_year'] - master_df['age_band'].str.split('[-+]', regex=True).str[0].astype(int)

yob = from_yob.astype(str).str.cat(to_yob.astype(str), '-')

yob_map = {
    '1891-1926': 'pre-1927',
    '1901-1926': 'pre-1927',
}

master_df['yob'] = yob.apply(lambda x: yob_map.get(x, x))

In [44]:
df_2001['census_year'] = 2001

# Map sex values

In [45]:
master_df['sex'] = master_df['sex'].map({'Females': 'female', 'Males': 'male', 'Female': 'female', 'Male': 'male'})

# Standardize column names

In [46]:
master_df.rename(columns={'geography code': 'geo_code', 'value': 'population'}, inplace=True)

In [47]:
# master_df.columns

In [48]:
master_df = master_df[['census_year', 'geo_code', 'geography', 'sex', 'religion', 'age', 'age_band', 'yob', 'population']]

# Join 2001 data

In [49]:
df_2001['geo_code'] = df_2001['geo_code'].replace(gc_map)

In [50]:
set(df_2001['geo_code']) ^ set(master_df['geo_code'])

set()

In [51]:
master_df = pd.concat([master_df, df_2001])

In [52]:
master_df['religion'] = master_df['religion'].replace({'Religion not stated': 'Not answered'})

# Write to disk

In [53]:
master_df.to_csv('../processed_data/england_wales.csv', index=False)